**Component buildup method to evaluate the parasitic drag coefficient $C_{D0}$**

To be done :
- accurate fuselage wetted area
- surface fraction under laminar flow
- landing gear drag coefficient
- aft fuselage upsweep angle drag
- protuberance drag

In [2]:
import numpy as np

- Input data

In [3]:
# FLIGHT CONDITIONS
V = 18.0 # m/s

# FUSELAGE
fus_width = 0.25
fus_height = 0.25
fus_length = 2.2
fus_x_tr = 0.0 # laminar to turbulent transition point
fus_d_eq = np.sqrt(4*fus_height*fus_width/np.pi) # equivalent diameter

# WING
w_t_c = 0.125 # wing thickness-to-chord ratio
w_x_c_max_t = 0.237 # wing max thickness location
w_chord = 0.50
w_S = 2.0
w_x_tr = 0.10 # laminar to turbulent transition point

# HORIZONTAL TAILPLANE
ht_t_c = 0.12
ht_x_c_max_t = 0.3
ht_chord = 0.41
ht_S = 0.66
ht_x_tr = 0.10

# VERTICAL TAILPLANE
vt_t_c = 0.12
vt_x_c_max_t = 0.3
vt_chord = 0.43
vt_S = 0.37
vt_x_tr = 0.10

# SKIN ROUGHNESS
skin_roughness = 0.634e-5 # [metre], for smooth paint (Raymer)

- Useful functions

In [4]:
# flat plate laminar friction coefficient
def C_f_lam(Re):
    return 1.328/np.sqrt(Re)

# flat plate turbulent friction coefficient
def C_f_turb(Re):
    return 0.0442/(Re**(1/6))

# Reynolds number at sea level conditions
def Re(V, l):
    return 1.225*V*l/1.8e-5

# cutoff Reynolds number
def Re_cutoff (l, k):
    return 38.21 * (l/k)**1.053

# lifting surface, pylon, strut, form factor
def FF_wing(x_c_max, t_c):
    return 1 + 0.6/x_c_max * t_c + 100 * t_c**4

# fuselage form factor
def FF_fus(length, d_eq, geometry: str='circ', ):
    f = length/d_eq
    FF = 0.9 + 5/f**1.5 + f/400
    if geometry == 'square':
        return FF*1.35 # Raymer : 30-40% extra FF for square-sided fuselage
    elif geometry == 'circ':
        return FF
    else :
        return FF

# wing wetted area
def S_wet_wing(S_exposed, t_c):
    return S_exposed * (1.977 + 0.52*t_c)


- Friction drag

In [5]:
# WING

w_Re_cutoff = Re_cutoff(w_chord, skin_roughness)
w_Re = Re(V, w_chord)

if w_Re_cutoff < w_Re:
    w_Re = w_Re_cutoff

# flat plate friction coefficient
w_C_f = w_x_tr * C_f_lam(w_Re) + (1-w_x_tr) * C_f_turb(w_Re)

# wing form factor
w_FF = FF_wing(w_x_c_max_t, w_t_c)

# wing interference factor
w_Q = 1.0 # Raymer, for a high-mounted wing

# wing wetted area
w_S_wet = S_wet_wing(w_S - fus_width * w_chord, w_t_c)

# wing parasitic drag surface
w_S_D_0 = w_C_f * w_FF * w_Q * w_S_wet


In [6]:
# HORIZONTAL TAILPLANE

ht_Re_cutoff = Re_cutoff(ht_chord, skin_roughness)
ht_Re = Re(V, ht_chord)

if ht_Re_cutoff < ht_Re:
    ht_Re = ht_Re_cutoff

# flat plate friction coefficient
ht_C_f = ht_x_tr * C_f_lam(ht_Re) + (1-ht_x_tr) * C_f_turb(ht_Re)

# ht form factor
ht_FF = FF_wing(ht_x_c_max_t, ht_t_c)

# ht interference factor
ht_Q = 1.05 # Raymer, for a conventional tail

# ht wetted area
ht_S_wet = S_wet_wing(ht_S, ht_t_c)

# ht parasitic drag surface
ht_S_D_0 = ht_C_f * ht_FF * ht_Q * ht_S_wet

In [7]:
# VERTICAL TAILPLANE

vt_Re_cutoff = Re_cutoff(vt_chord, skin_roughness)
vt_Re = Re(V, vt_chord)

if vt_Re_cutoff < vt_Re:
    vt_Re = vt_Re_cutoff

# flat plate friction coefficient
vt_C_f = vt_x_tr * C_f_lam(vt_Re) + (1-vt_x_tr) * C_f_turb(vt_Re)

# vt form factor
vt_FF = FF_wing(vt_x_c_max_t, vt_t_c)

# vt interference factor
vt_Q = 1.05 # Raymer, for a conventional tail

# vt wetted area
vt_S_wet = S_wet_wing(vt_S, vt_t_c)

# vt parasitic drag surface
vt_S_D_0 = vt_C_f * vt_FF * vt_Q * vt_S_wet

In [8]:
# FUSELAGE

fus_Re_cutoff = Re_cutoff(fus_d_eq, skin_roughness)
fus_Re = Re(V, fus_d_eq)

if fus_Re_cutoff < fus_Re:
    fus_Re = fus_Re_cutoff

# flat plate friction coefficient
fus_C_f = fus_x_tr * C_f_lam(fus_Re) + (1-fus_x_tr) * C_f_turb(fus_Re)

# fuselage form factor
fus_FF = FF_fus(fus_length, fus_d_eq, 'rect')

# fuselage interference factor
fus_Q = 1.0 # Raymer

# fuselage wetted area
fus_S_wet = 2 * (fus_width + fus_height) * fus_length * 0.8 # /!\ 0.8 factor is to be adjusted !!!!

# fuselage parasitic drag surface
fus_S_D_0 = fus_C_f * fus_FF * fus_Q * fus_S_wet

In [10]:
# PARASITIC FRICTION DRAG COEFFICIENT
C_D_0_f = (w_S_D_0 + ht_S_D_0 + vt_S_D_0 + fus_S_D_0) / w_S

- Miscellaneous drag

In [ ]:
# Landing gear
lg_